In [ ]:
import numpy as np
from numpy.linalg import norm

from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve
from scipy.optimize import minimize_scalar

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
from gpe_algorithms import inverse_iteration, dampened_inverse_iteration, shifted_inverse_iteration, calculate_energy
from gpe_problem import get_L_M, get_M, A_v

# Plots

## Parameters

In [ ]:
max_steps = 200
tol = 1e-8
bound = 8.
steps = 200
betas = np.array([1e-1, 1e0, 1e+1, 1e+2, 1e+3])

## Precomputations

In [ ]:
h = 2 * bound / steps
h_inv_sq = 1 / (h * h)
L_M = get_L_M(bound, steps, h_inv_sq)

# Potential V. Needed seperately for dampening
V = get_M(bound, steps)

## Helper methods for visualization

In [ ]:
def plot_result(axes, taken_steps, residuum, energy, diffs, beta, color, label=None):
    """
    Plot the results of a successful iteration on a 2x2 axes grid.

    Parameters
    ----------
    axes : ndarray of matplotlib.axes.Axes
        2x2 array of axes for plotting.
    taken_steps : int
        Number of steps taken in the iteration.
    residuum : ndarray
        Residuum values at each step.
    energy : ndarray
        Energy values at each step.
    diffs : ndarray
        Norm differences between consecutive iterates.
    beta : float
        Nonlinearity parameter used (for labeling).
    color : str
        Color for the plots.
    label : str, optional
        Custom label for the legend; if None, defaults to `beta={beta}`.
    """
    pt_range = np.arange(taken_steps)
    # Only add label once
    if label is None:
        axes[0,0].plot(pt_range, residuum[:taken_steps], label=f"beta={beta}", c=color, marker='o')
    else:
        axes[0,0].plot(pt_range, residuum[:taken_steps], label=label, c=color, marker='o')

    axes[0,1].plot(pt_range, diffs[:taken_steps], c=color, marker='o')
    axes[1,0].plot(pt_range, energy[:taken_steps], c=color, marker='o')

    pt_range = pt_range[:-1]
    energy_diff = energy[:taken_steps-1] - energy[1:taken_steps]
    axes[1,1].plot(pt_range, np.full_like(pt_range, 0), color='gray', linestyle='-.')
    axes[1,1].plot(pt_range, energy_diff, c=color, marker='o')


def plot_failed_result(axes, taken_steps, energy, diffs, beta, color):
    """
    Plot the results of a failed iteration on a 2x2 axes grid (no residuum available).

    Parameters
    ----------
    axes : ndarray of matplotlib.axes.Axes
        2x2 array of axes for plotting.
    taken_steps : int
        Number of steps taken in the iteration.
    energy : ndarray
        Energy values at each step.
    diffs : ndarray
        Norm differences between consecutive iterates.
    beta : float
        Nonlinearity parameter used (for labeling).
    color : str
        Color for the plots.
    """
    pt_range = np.arange(taken_steps)
    
    axes[0,1].plot(pt_range, diffs[:taken_steps], label=f"beta={beta}", c=color, marker='o')
    axes[1,0].plot(pt_range, energy[:taken_steps], c=color, marker='o')

    pt_range = pt_range[:-1]
    energy_diff = energy[:taken_steps-1] - energy[1:taken_steps]
    axes[1,1].plot(pt_range, np.full_like(pt_range, 0), color='gray', linestyle='-.')
    axes[1,1].plot(pt_range, energy_diff, c=color, marker='o')


def config_result_plot(fig, axes):
    """
    Configure a 2x2 axes grid for plotting iteration results.

    Sets log scales for x and/or y where appropriate, adds labels, and
    attaches a figure legend.

    Parameters
    ----------
    fig : matplotlib.figure.Figure
        Figure object containing the axes.
    axes : ndarray of matplotlib.axes.Axes
        2x2 array of axes to configure.
    """
    axes[0,0].set_yscale('log')
    axes[0,0].set_xscale('log')
    axes[0,0].set_ylabel('Residuum')
    axes[0,0].set_xlabel('Steps')
    
    axes[0,1].set_yscale('log')
    axes[0,1].set_xscale('log')
    axes[0,1].set_ylabel('Differences $\Vert u^n - u^{n-1} \Vert$')
    axes[0,1].set_xlabel('Steps')
    
    axes[1,0].set_yscale('linear')
    axes[1,0].set_xscale('log')
    axes[1,0].set_ylabel('Energy $E(u^n)$')
    axes[1,0].set_xlabel('Steps')
    
    axes[1,1].set_yscale('linear')
    axes[1,1].set_xscale('log')
    axes[1,1].set_ylabel('Energy diff $E(u^{n-1}) - E(u^n)$')
    axes[1,1].set_xlabel('Steps')
    
    fig.legend()

## Helper function to run the inverse iteration algorithms and remove duplicate code

In [ ]:
def run_algo(algo, file_name):
    """
    Run an inverse iteration algorithm (normal, shifted, or dampened) for multiple beta values,
    and plot the results on a 2x2 grid, saving the figure to `file_name`.

    Parameters
    ----------
    algo : function
        Iteration algorithm to run. Must accept the following parameters:
        - A : lambda taking vector v and returning a sparse matrix
        - calc_E : lambda taking vector v and returning energy
        - dim : int, problem dimension
        - max_steps : int, maximum iterations
        - tol : float, convergence tolerance
    file_name : str
        Path to save the figure.
    """
    colors = ["darkgreen", "gold", "orange", "red", "darkred"]

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))

    for i, beta in enumerate(betas):
        # Define matrix operator and energy functional for this beta
        A = lambda v: A_v(L_M, v, beta, h_inv_sq)
        calc_E = lambda v: calculate_energy(v, V, beta, h, steps)

        print(f"Starting iteration with bound={bound}, steps={steps}, beta={beta}")
        
        success, taken_steps, last, residuum, energy, diffs = algo(
            A, calc_E, steps * steps, max_steps, tol
        )

        if success:
            eigenval = last.T @ A(last) @ last / (last.T @ last)
            print(f"Successful iteration: converged in {taken_steps + 1} steps "
                  f"to eigenvalue ≈ {eigenval:.4f}")
            plot_result(axes, taken_steps, residuum, energy, diffs, beta, colors[i])
        else:
            print(f"Iteration failed after {max_steps} steps for beta={beta}!")
            plot_failed_result(axes, max_steps, energy, diffs, beta, 'black')

    config_result_plot(fig, axes)
    plt.tight_layout()
    plt.savefig(file_name, dpi=300)
    plt.show()

## Testing

In [ ]:
run_algo(inverse_iteration, 'gpe-inverse-iteration.png')

In [ ]:
run_algo(dampened_inverse_iteration, 'gpe-inverse-iteration-dampened.png')

In [ ]:
run_algo(shifted_inverse_iteration, 'gpe-inverse-iteration-shifted.png')

## Comparision dampened - undampened

In [ ]:
d_colors = ["darkgreen", "forestgreen", "green", "limegreen", "springgreen"]
u_colors = ["gold", "orange", "darkorange", "red", "darkred"]

fig, axes = plt.subplots(3, 2, figsize=(12, 12))
axes = np.array(axes)  # ensure indexing works

# Map axes for normal (u) and dampened (d) iterations
u_axes = axes[[0,2], :]
d_axes = axes[[1,2], :]  # only row 1

for i, beta in enumerate(betas):
    A = lambda v: A_v(L_M, v, beta, h_inv_sq)
    calc_E = lambda v: calculate_energy(v, V, beta, h, steps)

    print(f"Starting with bound={bound}, steps={steps}, beta={beta}")

    u_success, u_taken_steps, u_last, u_residuum, u_energy, u_diffs = inverse_iteration(A, calc_E, steps**2, max_steps, tol)
    d_success, d_taken_steps, d_last, d_residuum, d_energy, d_diffs = dampened_inverse_iteration(A, calc_E, steps**2, max_steps, tol)

    if u_success and d_success:
        print(f"Normal iteration converged in {u_taken_steps+1} steps; dampened in {d_taken_steps+1}")
        print(f"Norm difference between results: {norm(u_last - d_last):.4e}")
        plot_result(u_axes, u_taken_steps, u_residuum, u_energy, u_diffs, beta, u_colors[i], label=f"beta={beta} (normal)")
        plot_result(d_axes, d_taken_steps, d_residuum, d_energy, d_diffs, beta, d_colors[i], label=f"beta={beta} (dampened)")
    else:
        print(f"Iteration failed: normal={u_success}, dampened={d_success}")
        if u_success:
            plot_result(u_axes, u_taken_steps, u_residuum, u_energy, u_diffs, beta, u_colors[i], label=f"beta={beta} (normal)")
        if d_success:
            plot_result(d_axes, d_taken_steps, d_residuum, d_energy, d_diffs, beta, d_colors[i], label=f"beta={beta} (dampened)")

config_result_plot(fig, u_axes)
config_result_plot(fig, d_axes)

plt.tight_layout()
plt.savefig('gpe-inverse-iteration-comparision.png', dpi=300)
plt.show()